# Edge TPU Object Detection Training Script

1) Prepare Dataset from etc -> kitti -> TFRecord

2) Train fp32 TF model

3) Optimize TF model into TF-lite model

4) Convert TF-lite model into quantized edgetpu model


In [1]:
# Setting up env variables for cleaner command line commands

# constants.sh
#!declare -A ckpt_link_map
#!declare -A ckpt_name_map
#!declare -A config_filename_map
#
#!ckpt_link_map["mobilenet_v1_ssd"]="http://download.tensorflow.org/models/object_detection/ssd_mobilenet_v1_quantized_300x300_coco14_sync_2018_07_18.tar.gz"
#!ckpt_link_map["mobilenet_v2_ssd"]="http://download.tensorflow.org/models/object_detection/ssd_mobilenet_v2_quantized_300x300_coco_2019_01_03.tar.gz"
#
#!ckpt_name_map["mobilenet_v1_ssd"]="ssd_mobilenet_v1_quantized_300x300_coco14_sync_2018_07_18"
#!ckpt_name_map["mobilenet_v2_ssd"]="ssd_mobilenet_v2_quantized_300x300_coco_2019_01_03"
#
#!config_filename_map["mobilenet_v1_ssd-true"]="pipeline_mobilenet_v1_ssd_retrain_whole_model.config"
#!config_filename_map["mobilenet_v1_ssd-false"]="pipeline_mobilenet_v1_ssd_retrain_last_few_layers.config"
#!config_filename_map["mobilenet_v2_ssd-true"]="pipeline_mobilenet_v2_ssd_retrain_whole_model.config"
#!config_filename_map["mobilenet_v2_ssd-false"]="pipeline_mobilenet_v2_ssd_retrain_last_few_layers.config"


%env INPUT_TENSORS='normalized_input_image_tensor'
%env OUTPUT_TENSORS='TFLite_Detection_PostProcess,TFLite_Detection_PostProcess:1,TFLite_Detection_PostProcess:2,TFLite_Detection_PostProcess:3'

%env TF_BASE=/works/tensorflow/models/research
%env DATA_DIR=/DL_data/pig500_20200303_sungil
%env WORK_DIR=/works/tensorflow/train_detect
%env TFREC_DIR=/works/tensorflow/train_detect/learn/tfrecord
%env CKPT_DIR=/works/tensorflow/train_detect/learn/ckpt
%env TRAIN_DIR=/works/tensorflow/train_detect/learn/train
%env OUTPUT_DIR=/works/tensorflow/train_detect/learn/models

#The Last few layers
NUM_TRAINING_STEPS=500
NUM_EVAL_STEPS=100

#Whole Training
##NUM_TRAINING_STEPS=50000
##NUM_EVAL_STEPS=2000


env: INPUT_TENSORS='normalized_input_image_tensor'
env: OUTPUT_TENSORS='TFLite_Detection_PostProcess,TFLite_Detection_PostProcess:1,TFLite_Detection_PostProcess:2,TFLite_Detection_PostProcess:3'
env: TF_BASE=/works/tensorflow/models/research
env: DATA_DIR=/DL_data/pig500_20200303_sungil
env: WORK_DIR=/works/tensorflow/train_detect
env: TFREC_DIR=/works/tensorflow/train_detect/learn/tfrecord
env: CKPT_DIR=/works/tensorflow/train_detect/learn/ckpt
env: TRAIN_DIR=/works/tensorflow/train_detect/learn/train
env: OUTPUT_DIR=/works/tensorflow/train_detect/learn/models


In [2]:
# Generate kitti data format
!python ./src/matching_list_maker.py --data_path $DATA_DIR
!python ./src/json_to_csv.py --data_path $DATA_DIR --class_num 3 --class_list "pig_s,pig_l,pig_part"
!python ./src/csv2KITTI.py --data_path $DATA_DIR
!mkdir -p $DATA_DIR/kitti
!cp -R $DATA_DIR/img $DATA_DIR/kitti/image_2
!cp -R $DATA_DIR/lab_kitti_train $DATA_DIR/kitti/label_2

/DL_data/pig500_20200303_sungil
499it [00:00, 965.83it/s] 
499it [00:02, 173.95it/s]


In [3]:
# Check data are valid
!echo $DATA_DIR/kitti/image_2
!ls -al $DATA_DIR/kitti/image_2
!ls -al $DATA_DIR/kitti/label_2

/DL_data/pig500_20200303_sungil/kitti/image_2
total 54468
drwxr-xr-x 1 root root  29946 Apr 22 07:06 .
drwxr-xr-x 1 root root     28 Apr 22 06:54 ..
-rw-r--r-- 1 root root 134374 Apr 22 06:54 2020-03-01_08-39-07_1ch_00.jpg
-rw-r--r-- 1 root root 123130 Apr 22 06:54 2020-03-01_08-39-07_2ch_00.jpg
-rw-r--r-- 1 root root 132919 Apr 22 06:54 2020-03-01_08-39-07_3ch_00.jpg
-rw-r--r-- 1 root root 176676 Apr 22 06:54 2020-03-01_08-39-07_4ch_00.jpg
-rw-r--r-- 1 root root 125168 Apr 22 06:54 2020-03-01_08-39-07_6ch_00.jpg
-rw-r--r-- 1 root root 117791 Apr 22 06:54 2020-03-01_08-39-07_7ch_00.jpg
-rw-r--r-- 1 root root 127570 Apr 22 06:54 2020-03-01_08-39-07_8ch_00.jpg
-rw-r--r-- 1 root root 132661 Apr 22 06:54 2020-03-01_09-00-13_1ch_00.jpg
-rw-r--r-- 1 root root 127052 Apr 22 06:54 2020-03-01_09-00-13_2ch_00.jpg
-rw-r--r-- 1 root root 129913 Apr 22 06:54 2020-03-01_09-00-13_3ch_00.jpg
-rw-r--r-- 1 root root 178239 Apr 22 06:54 2020-03-01_09-00-13_4ch_00.jpg
-rw-r--r-- 1 root root 130453 Apr 22 

total 5648
drwxr-xr-x 1 root root 29970 Apr 22 07:06 .
drwxr-xr-x 1 root root    28 Apr 22 06:54 ..
-rw-r--r-- 1 root root  9245 Apr 22 06:54 2020-03-01_08-39-07_1ch_00.txt
-rw-r--r-- 1 root root  5415 Apr 22 06:54 2020-03-01_08-39-07_2ch_00.txt
-rw-r--r-- 1 root root  9045 Apr 22 06:54 2020-03-01_08-39-07_3ch_00.txt
-rw-r--r-- 1 root root  9040 Apr 22 06:54 2020-03-01_08-39-07_4ch_00.txt
-rw-r--r-- 1 root root 10440 Apr 22 06:54 2020-03-01_08-39-07_6ch_00.txt
-rw-r--r-- 1 root root  9475 Apr 22 06:54 2020-03-01_08-39-07_7ch_00.txt
-rw-r--r-- 1 root root  9295 Apr 22 06:54 2020-03-01_08-39-07_8ch_00.txt
-rw-r--r-- 1 root root  9820 Apr 22 06:54 2020-03-01_09-00-13_1ch_00.txt
-rw-r--r-- 1 root root  5730 Apr 22 06:54 2020-03-01_09-00-13_2ch_00.txt
-rw-r--r-- 1 root root  9910 Apr 22 06:54 2020-03-01_09-00-13_3ch_00.txt
-rw-r--r-- 1 root root 10940 Apr 22 06:54 2020-03-01_09-00-13_4ch_00.txt
-rw-r--r-- 1 root root  8560 Apr 22 06:54 2020-03-01_09-00-13_6ch_00.txt
-rw-r--r-- 1 root root 1

In [3]:
#Generate TFRecord
!mkdir -p $TFREC_DIR
!cd $TF_BASE
!python $WORK_DIR/src/create_kitti_tf_record.py \
        --classes_to_use 'pig_s,pig_l' \
        --data_dir "${DATA_DIR}/kitti" \
        --label_map_path "${DATASET_DIR}/kitti_label_map.pbtxt" \
        --output_path "${DATASET_DIR}/kitti" \
        --validation_set_size 50



Traceback (most recent call last):
  File "/works/tensorflow/train_detect/src/create_kitti_tf_record.py", line 47, in <module>
    from object_detection.utils import dataset_util
ImportError: No module named object_detection.utils


* Please change 'pipeline.config' manually considering purpose of this training.

In [1]:
#Retrain detection model

!mkdir "${TRAIN_DIR}"

!python object_detection/model_main.py \
  --pipeline_config_path="${CKPT_DIR}/pipeline.config" \
  --model_dir="${TRAIN_DIR}" \
  --num_train_steps="${NUM_TRAINING_STEPS}" \
  --num_eval_steps="${NUM_EVAL_STEPS}"


SyntaxError: invalid syntax (<ipython-input-1-5333d064cf9d>, line 1)

In [ ]:
# "EXPORTING frozen graph from checkpoint..."

!python object_detection/export_tflite_ssd_graph.py \
  --pipeline_config_path="${CKPT_DIR}/pipeline.config" \
  --trained_checkpoint_prefix="${TRAIN_DIR}/model.ckpt-${ckpt_number}" \
  --output_directory="${OUTPUT_DIR}" \
  --add_postprocessing_op=true

In [ ]:
# "CONVERTING frozen graph to TF Lite file..."

!tflite_convert \
  --output_file="${OUTPUT_DIR}/output_tflite_graph.tflite" \
  --graph_def_file="${OUTPUT_DIR}/tflite_graph.pb" \
  --inference_type=QUANTIZED_UINT8 \
  --input_arrays="${INPUT_TENSORS}" \
  --output_arrays="${OUTPUT_TENSORS}" \
  --mean_values=128 \
  --std_dev_values=128 \
  --input_shapes=1,300,300,3 \
  --change_concat_input_ranges=false \
  --allow_nudging_weights_to_use_fast_gemm_kernel=true \
  --allow_custom_ops